<a href="https://colab.research.google.com/github/Minhvt34/cbm_codes_open/blob/master/Transformer_scratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/oliverguhr/transformer-time-series-prediction.git


Cloning into 'transformer-time-series-prediction'...
remote: Enumerating objects: 35, done.
remote: Counting objects: 100% (35/35), done.
remote: Compressing objects: 100% (25/25), done.
remote: Total 35 (delta 13), reused 23 (delta 6), pack-reused 0
Unpacking objects: 100% (35/35), done.


In [1]:
import torch
import torch.nn as nn
import numpy as np
import time
import math
from matplotlib import pyplot


In [2]:
torch.manual_seed(0)
np.random.seed(0)

In [ ]:
# S is the  source sequence length
# T is the target sequence length
# N is the batch size
# E is the feature number

#src = torch.rand((10, 32, 512)) (S, N, E)
#tgt = torch.rand((20, 32, 512)) (T, N, E)
#out = transformer_model(src, tgt)

In [3]:
import os 
import numpy as np
import random
import math
import json
from functools import partial

import matplotlib.pyplot as plt
plt.set_cmap('cividis')
%matplotlib inline
from IPython.display import set_matplotlib_formats
set_matplotlib_formats('svg', 'pdf')
from matplotlib.colors import to_rgb
import matplotlib
matplotlib.rcParams['lines.linewidth'] = 2.0

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data as data
import torch.optim as optim

In [4]:
try:
  import pytorch_lightning as pl

except ModuleNotFoundError:
  !pip install --quiet pytorch-lightning>=1.4
  import pytorch_lightning as pl

from pytorch_lightning.callbacks import LearningRateMonitor, ModelCheckpoint

In [4]:
input_window = 100 # number of input steps
output_window = 5 # number of prediction steps, in this model its fixed to one
batch_size = 64
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [1]:
def scaled_dot_product(q, k, v, mask=None):
  d_k = q.size()[-1]
  attn_logits = torch.matmul(q, k.transpose(-2, -1))
  attn_logits = attn_logits/math.sqrt(d_k)
  if mask is not None:
    attn_logits = attn_logits.masked_fill(mask == 0, -9e15)
  attention = F.softmax(attn_logits, dim=-1)
  values = torch.matmul(attention, v)
  return values, attention

In [5]:
seq_len, d_k = 3, 2
pl.seed_everything(42)
q = torch.randn(seq_len, d_k)
k = torch.randn(seq_len, d_k)
v = torch.randn(seq_len, d_k)

values, attention = scaled_dot_product(q, k, v)
print("Q\n", q)
print("K\n", k)
print("V\n", v)
print("Values\n", values)
print("Attention\n", attention)

Global seed set to 42


Q
 tensor([[ 0.3367,  0.1288],
        [ 0.2345,  0.2303],
        [-1.1229, -0.1863]])
K
 tensor([[ 2.2082, -0.6380],
        [ 0.4617,  0.2674],
        [ 0.5349,  0.8094]])
V
 tensor([[ 1.1103, -1.6898],
        [-0.9890,  0.9580],
        [ 1.3221,  0.8172]])
Values
 tensor([[ 0.5698, -0.1520],
        [ 0.5379, -0.0265],
        [ 0.2246,  0.5556]])
Attention
 tensor([[0.4028, 0.2886, 0.3086],
        [0.3538, 0.3069, 0.3393],
        [0.1303, 0.4630, 0.4067]])


In [7]:
class MultiheadAttention(nn.Module):

  def __init__(self, input_dim, embed_dim, num_heads):
      super().__init__()
      assert embed_dim % num_heads == 0, "Embedding dimension must be 0 modulo number of heads"

      self.embed_dim = embed_dim
      self.num_heads = num_heads
      self.head_dim = embed_dim // num_heads

      self.qkv_proj = nn.Linear(input_dim, 3*embed_dim)
      self.o_proj = nn.Linear(embed_dim, embed_dim)

      self._reset_parameters()

  def _reset_parameters(self):
      # Original Transformer initialization
      nn.init.xavier_normal_(self.qkv_proj.weight)
      self.qkv_proj.bias.data.fill_(0)
      nn.init.xavier_normal_(self.o_proj.weight)
      self.o_proj.bias.data.fill_(0)

  def forward(self, x, mask=None, return_attention=False):
    batch_size, seq_lenght, embed_dim = x.size()
    qkv = self.qkv_proj(x)

    # Separate Q, K, V from linear output
    qkv = qkv.reshape(batch_size, seq_lenght, self.num_heads, 3*self.head_dim)
    qkv = qkv.permute(0, 2, 1, 3) # [Batch, Head, Seqlen, Dims]
    q, k, v = qkv.chunk(3, dim=-1)

    #Determine value outputs
    values, attention = scaled_dot_product(q, k, v, mask=mask)
    values = values.permute(0, 2, 1, 3) #[Batch, SeqLen, Head, Dims]
    values = values.reshape(batch_size, seq_lenght, embed_dim)
    o = self.o_proj(values)

    if return_attention:
      return o, attention
    else:
      return o

In [5]:
class PositionalEncoding(nn.Module):

    def __init__(self, d_model, max_len=100):
        super(PositionalEncoding, self).__init__()       
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0).transpose(0, 1)
        #pe.requires_grad = False
        self.register_buffer('pe', pe)

    def forward(self, x):
        # print(x[0,0,0])
        # print(self.pe[0,0,0])
        # print(x.size(1), x.size(0))
        # print(self.pe[:x.size(0), :].shape)
        #x + self.pe[:x.size(0), :]
        #x = x + self.pe[:, :x.size(0)]
        #ts = x + self.pe[:x.size(0), :]
        #print("after: ", ts[0,0,0])
        return x + self.pe[:x.size(0), :]

In [6]:
class TransAm(nn.Module):
    def __init__(self,feature_size=250,num_layers=1,dropout=0.1):
        super(TransAm, self).__init__()
        self.model_type = 'Transformer'
        
        self.src_mask = None
        self.pos_encoder = PositionalEncoding(feature_size)
        self.encoder_layer = nn.TransformerEncoderLayer(d_model=feature_size, nhead=10, dropout=dropout)
        self.transformer_encoder = nn.TransformerEncoder(self.encoder_layer, num_layers=num_layers)        
        self.decoder = nn.Linear(feature_size,1)
        self.init_weights()

    def init_weights(self):
        initrange = 0.1    
        self.decoder.bias.data.zero_()
        self.decoder.weight.data.uniform_(-initrange, initrange)

    def forward(self,src):
        if self.src_mask is None or self.src_mask.size(0) != len(src):
            device = src.device
            mask = self._generate_square_subsequent_mask(len(src)).to(device)
            self.src_mask = mask

        src = self.pos_encoder(src)
        output = self.transformer_encoder(src,self.src_mask)#, self.src_mask)
        output = self.decoder(output)
        return output

    def _generate_square_subsequent_mask(self, sz):
        mask = (torch.triu(torch.ones(sz, sz)) == 1).transpose(0, 1)
        mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
        return mask

In [7]:
# if window is 100 and prediction step is 1
# in -> [0..99]
# target -> [1..100]

def create_inout_sequences(input_data, tw):
    inout_seq = []
    L = len(input_data)
    for i in range(L-tw):
        train_seq = np.append(input_data[i:i+tw][:-output_window] , output_window * [0])
        train_label = input_data[i:i+tw]
        #train_label = input_data[i+output_window:i+tw+output_window]
        inout_seq.append((train_seq ,train_label))
    return torch.FloatTensor(inout_seq)

In [8]:
from pandas import read_csv
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

In [8]:
col_names = ['b1_ch1', 'b1_ch2', 'b2_ch3', 'b2_ch4', 'b3_ch5', 'b3_ch6', 'b4_ch7', 'b4_ch8']
df = pd.read_csv('/content/drive/MyDrive/2003.10.22.12.06.24', sep='\t', names=col_names)
series = df['b1_ch1']

# looks like normalizing input values curtial for the model
scaler = MinMaxScaler(feature_range=(-1, 1)) 
amplitude = scaler.fit_transform(series.to_numpy().reshape(-1, 1)).reshape(-1)
#amplitude = scaler.fit_transform(amplitude.reshape(-1, 1)).reshape(-1)

samples = 20000
train_data = amplitude[:samples]
test_data = amplitude[samples:]
test_data.shape

(480,)

In [9]:
def get_data():
  # time = np.arange(0, 10000, 0.1)
  # amplitude = np.sin(time) + np.sin(time*0.05) + np.sin(time*0.12)*np.random.normal(-0.2, 0.2, len(time))

  #loading weather data from a file
  #series = read_csv('/content/2003.10.22.12.06.24', header=0, index_col=0, parse_dates=True, squeeze=True)
  col_names = ['b1_ch1', 'b1_ch2', 'b2_ch3', 'b2_ch4', 'b3_ch5', 'b3_ch6', 'b4_ch7', 'b4_ch8']
  df = pd.read_csv('/content/drive/MyDrive/2003.10.22.12.06.24', sep='\t', names=col_names)
  series = df['b1_ch1']
  
  # looks like normalizing input values curtial for the model
  scaler = MinMaxScaler(feature_range=(-1, 1)) 
  amplitude = scaler.fit_transform(series.to_numpy().reshape(-1, 1)).reshape(-1)
  #amplitude = scaler.fit_transform(amplitude.reshape(-1, 1)).reshape(-1)

  samples = 20000
  train_data = amplitude[:samples]
  test_data = amplitude[samples:]

  # convert our train data into a pytorch train tensor
  #train_tensor = torch.FloatTensor(train_data).view(-1)
  # todo: add comment.. 
  train_sequence = create_inout_sequences(train_data,input_window)
  train_sequence = train_sequence[:-output_window] #todo: fix hack? -> din't think this through, looks like the last n sequences are to short, so I just remove them. Hackety Hack.. 

  #test_data = torch.FloatTensor(test_data).view(-1) 
  test_data = create_inout_sequences(test_data,input_window)
  test_data = test_data[:-output_window] #todo: fix hack?

  return train_sequence.to(device),test_data.to(device)


In [10]:
def get_batch(source, i,batch_size):
    seq_len = min(batch_size, len(source) - 1 - i)
    data = source[i:i+seq_len]    
    input = torch.stack(torch.stack([item[0] for item in data]).chunk(input_window,1)) # 1 is feature size
    target = torch.stack(torch.stack([item[1] for item in data]).chunk(input_window,1))
    return input, target

In [11]:
from torch.optim import optimizer

In [12]:
def train(train_data):
    model.train() # Turn on the train mode
    total_loss = 0.
    start_time = time.time()

    for batch, i in enumerate(range(0, len(train_data) - 1, batch_size)):
        data, targets = get_batch(train_data, i,batch_size)
        optimizer.zero_grad()
        output = model(data)        

        if calculate_loss_over_all_values:
            loss = criterion(output, targets)
        else:
            loss = criterion(output[-output_window:], targets[-output_window:])
    
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.5)
        optimizer.step()

        total_loss += loss.item()
        log_interval = int(len(train_data) / batch_size / 5)
        if batch % log_interval == 0 and batch > 0:
            cur_loss = total_loss / log_interval
            elapsed = time.time() - start_time
            print('| epoch {:3d} | {:5d}/{:5d} batches | '
                  'lr {:02.6f} | {:5.2f} ms | '
                  'loss {:5.5f} | ppl {:8.2f}'.format(
                    epoch, batch, len(train_data) // batch_size, lr_scheduler.get_lr()[0],
                    elapsed * 1000 / log_interval,
                    cur_loss, math.exp(cur_loss)))
            total_loss = 0
            start_time = time.time()

In [13]:
calculate_loss_over_all_values = False

In [14]:
def plot_and_loss(eval_model, data_source,epoch):
    eval_model.eval() 
    total_loss = 0.
    test_result = torch.Tensor(0)    
    truth = torch.Tensor(0)
    with torch.no_grad():
        for i in range(0, len(data_source) - 1):
            data, target = get_batch(data_source, i,1)
            # look like the model returns static values for the output window
            output = eval_model(data)    
            if calculate_loss_over_all_values:                                
                total_loss += criterion(output, target).item()
            else:
                total_loss += criterion(output[-output_window:], target[-output_window:]).item()
            
            test_result = torch.cat((test_result, output[-1].view(-1).cpu()), 0) #todo: check this. -> looks good to me
            truth = torch.cat((truth, target[-1].view(-1).cpu()), 0)
            
    #test_result = test_result.cpu().numpy()
    len(test_result)

    pyplot.plot(test_result,color="red")
    pyplot.plot(truth[:20000],color="blue")
    pyplot.plot(test_result-truth,color="green")
    pyplot.grid(True, which='both')
    pyplot.axhline(y=0, color='k')
    pyplot.savefig('/content/transformer-epoch%d.png'%epoch)
    pyplot.close()
    
    return total_loss / i

In [15]:
# predict the next n steps based on the input data 
def predict_future(eval_model, data_source,steps):
    eval_model.eval() 
    total_loss = 0.
    test_result = torch.Tensor(0)    
    truth = torch.Tensor(0)
    _ , data = get_batch(data_source, 0,1)
    with torch.no_grad():
        for i in range(0, steps,1):
            input = torch.clone(data[-input_window:])
            input[-output_window:] = 0     
            output = eval_model(data[-input_window:])                        
            data = torch.cat((data, output[-1:]))
            
    data = data.cpu().view(-1)
    

    pyplot.plot(data,color="red")       
    pyplot.plot(data[:input_window],color="blue")
    pyplot.grid(True, which='both')
    pyplot.axhline(y=0, color='k')
    pyplot.savefig('/content/results/transformer-future%d.png'%steps)
    pyplot.close()

In [16]:
def evaluate(eval_model, data_source):
    eval_model.eval() # Turn on the evaluation mode
    total_loss = 0.
    eval_batch_size = 1000
    with torch.no_grad():
        for i in range(0, len(data_source) - 1, eval_batch_size):
            data, targets = get_batch(data_source, i,eval_batch_size)
            output = eval_model(data)            
            if calculate_loss_over_all_values:
                total_loss += len(data[0])* criterion(output, targets).cpu().item()
            else:                                
                total_loss += len(data[0])* criterion(output[-output_window:], targets[-output_window:]).cpu().item()            
    return total_loss / len(data_source)

In [17]:
import torch.optim as optim

In [18]:
class CosineWarmupScheduler(optim.lr_scheduler._LRScheduler):
    
    def __init__(self, optimizer, warmup, max_iters):
        self.warmup = warmup
        self.max_num_iters = max_iters
        super().__init__(optimizer)
        
    def get_lr(self):
        lr_factor = self.get_lr_factor(epoch=self.last_epoch)
        return [base_lr * lr_factor for base_lr in self.base_lrs]
    
    def get_lr_factor(self, epoch):
        lr_factor = 0.5 * (1 + np.cos(np.pi * epoch / self.max_num_iters))
        if epoch <= self.warmup:
            lr_factor *= epoch * 1.0 / self.warmup
        return lr_factor

In [ ]:
train_data, val_data = get_data()
model = TransAm().to(device)

criterion = nn.MSELoss()
#lr = 0.005 
optimizer = optim.Adam(model.parameters(), lr=1e-2)
lr_scheduler = CosineWarmupScheduler(optimizer=optimizer, warmup=50, max_iters=2000)

# optimizer = torch.optim.AdamW(model.parameters(), lr=lr)
# scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma=0.98)

best_val_loss = float("inf")
epochs = 100 # The number of epochs
best_model = None

for epoch in range(1, epochs + 1):
    epoch_start_time = time.time()
    train(train_data)
    
    if(epoch % 10 is 0):
        val_loss = plot_and_loss(model, val_data,epoch)
        predict_future(model, val_data,200)
    else:
        val_loss = evaluate(model, val_data)
   
    print('-' * 89)
    print('| end of epoch {:3d} | time: {:5.2f}s | valid loss {:5.5f} | valid ppl {:8.2f}'.format(epoch, (time.time() - epoch_start_time),
                                     val_loss, math.exp(val_loss)))
    print('-' * 89)

    #if val_loss < best_val_loss:
    #    best_val_loss = val_loss
    #    best_model = model
    lr_scheduler.step()

    #scheduler.step() 

#src = torch.rand(input_window, batch_size, 1) # (source sequence length,batch size,feature number) 
#out = model(src)
#
#print(out)
#print(out.shape)

In [ ]:
import torch
import torch.nn as nn


class WeibullLossRMSE(nn.Module):
    """
    y_hat       : predicted RUL
    y           : true RUL
    y_days      : true age (in days)
    lambda_mod  : lambda modifier
    eta         : characteristic life
    beta        : shape parameter for weibull
    """

    def __init__(self, eps=1e-8):
        super(WeibullLossRMSE, self).__init__()
        self.eps = eps


    def forward(self, y_hat, y, y_days, lambda_mod=2.0, eta=90.0, beta=2.0):

        y_hat_days = (y_days + y) - y_hat

        # remove any "inf" values from when divided by zero
        y_hat_days = y_hat_days[torch.isfinite(y_hat_days)]

        def weibull_cdf(t, eta, beta):
            "weibull CDF function"
            return 1.0 - torch.exp(-1.0 * ((t / eta) ** beta))

        cdf = weibull_cdf(y_days, eta, beta)
        cdf_hat = weibull_cdf(y_hat_days, eta, beta)

        return lambda_mod * torch.sqrt(torch.mean(cdf_hat - cdf) ** 2 + self.eps)


class WeibullLossRMSLE(nn.Module):
    """
    y_hat       : predicted RUL
    y           : true RUL
    y_days      : true age (in days)
    lambda_mod  : lambda modifier
    eta         : characteristic life
    beta        : shape parameter for weibull
    """

    def __init__(self, eps=1e-8):
        super(WeibullLossRMSLE, self).__init__()
        self.eps = eps


    def forward(self, y_hat, y, y_days, lambda_mod=2.0, eta=90.0, beta=2.0):

        y_hat_days = (y_days + y) - y_hat

        # remove any "inf" values from when divided by zero
        y_hat_days = y_hat_days[torch.isfinite(y_hat_days)]

        def weibull_cdf(t, eta, beta):
            "weibull CDF function"
            return 1.0 - torch.exp(-1.0 * ((t / eta) ** beta))

        cdf = weibull_cdf(y_days, eta, beta)
        cdf_hat = weibull_cdf(y_hat_days, eta, beta)

        return lambda_mod * torch.sqrt(torch.mean(torch.log(cdf_hat + 1) - torch.log(cdf+1)) ** 2 + self.eps)


class WeibullLossMSE(nn.Module):
    """
    y_hat       : predicted RUL
    y           : true RUL
    y_days      : true age (in days)
    lambda_mod  : lambda modifier
    eta         : characteristic life
    beta        : shape parameter for weibull
    """

    def __init__(self):
        super(WeibullLossMSE, self).__init__()


    def forward(self, y_hat, y, lambda_mod=2.0, eta=90.0, beta=2.0):

        y_hat_days = y - y_hat

        # remove any "inf" values from when divided by zero
        y_hat_days = y_hat_days[torch.isfinite(y_hat_days)]

        def weibull_cdf(t, eta, beta):
            "weibull CDF function"
            return 1.0 - torch.exp(-1.0 * ((t / eta) ** beta))

        cdf = weibull_cdf(y, eta, beta)
        cdf_hat = weibull_cdf(y_hat_days, eta, beta)

        return lambda_mod * torch.mean((cdf_hat - cdf) ** 2)


class RMSELoss(nn.Module):
    # https://discuss.pytorch.org/t/rmse-loss-function/16540/4

    def __init__(self, eps=1e-8):
        super(RMSELoss, self).__init__()
        self.mse = nn.MSELoss()
        self.eps = eps

    def forward(self, y_hat, y):
        return torch.sqrt(self.mse(y_hat, y) + self.eps)


class RMSLELoss(nn.Module):
    # https://discuss.pytorch.org/t/rmse-loss-function/16540/4

    def __init__(self, eps=1e-8):
        super(RMSLELoss, self).__init__()
        self.mse = nn.MSELoss()
        self.eps = eps

    def forward(self, y_hat, y):
        return torch.sqrt(self.mse(torch.log(y_hat + 1), torch.log(y + 1)) + self.eps)


class MAPELoss(nn.Module):

    def __init__(self, eps=1e-8):
        super(MAPELoss, self).__init__()
        self.eps = eps

    def forward(self, y_hat, y):

        return torch.mean(torch.abs(y - y_hat) / torch.abs(y + self.eps)) * 100